In [1]:
from azure.identity import DefaultAzureCredential

account_url = f"https://wboylesbackups.blob.core.windows.net"
credential = DefaultAzureCredential()

In [2]:
from azure.storage.blob import BlobClient

def download_blob(container_name: str, blob_name: str, out_filename: str | None = None):
    if out_filename is None:
        out_filename = container_name + blob_name

    blob_client = BlobClient(account_url, container_name, blob_name, credential=credential)

    with open(out_filename, "wb") as f:
        f.write(blob_client.download_blob().readall())

In [3]:
from azure.storage.blob import BlobPrefix, ContainerClient, BlobProperties

# Returns list of BlobProperties in preorder traversal
# Does not include folders, only files
# TODO: This does not properly show empty directories
def list_blobs(container_name: str, prefix: str="") -> list[BlobProperties]:
    #print(container_name, prefix)
    
    blobs: list[BlobProperties] = []
    container_client = ContainerClient(account_url, container_name, credential)

    for blob in container_client.walk_blobs(name_starts_with=prefix):
        name: str = blob.name # type: ignore
        
        if isinstance(blob, BlobPrefix):
            blobs += list_blobs(container_name, prefix=name)
        
        blobs.append(blob)


    return blobs

In [4]:
def create_folder(container_name: str, folder_name: str):
    container_client = ContainerClient(account_url, container_name, credential)

    # This is a hack, but it works better than using the data lake file system
    container_client.upload_blob(f"{folder_name}/_", "").delete_blob()

# blob_client.get_blob_properties().metadata['hdi_isfolder'] == 'true'

In [5]:
from azure.storage.blob import BlobClient

def upload_file(container_name: str, local_path: str, azure_path: str):
    blob_client = BlobClient(account_url, container_name, azure_path, credential=credential)
    with open(local_path, "rb") as f:
        blob_client.upload_blob(f)